In [1]:
import csv
import pandas as pd

df = pd.read_csv('content_token.csv')

In [2]:
df

,Unnamed: 0,date,content,category,rate_change,content_token
0,0,20050609,[edaily 김현동기자] 미국 뉴욕 맨해튼의 아파트 매매가격이 사상 최고가 행진을...,ED,0,"['부동산/NNG;가격/NNG;상승/NNG', '가격/NNG;상승/NNG;우호적/V..."
1,1,20050609,[edaily 하정민기자] 국제통화기금(IMF)이 유럽통화동맹(EMU) 체제 유지에...,ED,0,"['ecb/NNG;금리/NNG;인하/NNG', '경제/NNG;살/VV']"
2,2,20050609,[뉴욕=edaily 정명수특파원] 다우 지수가 어렵게 강보합선을 지켰다. 나스닥은 ...,ED,0,"['fed/NNG;금리/NNG;인상/NNG;경고/NNG', 'fed/NNG;장기/N..."
3,3,20050609,"[뉴욕=edaily 정명수특파원] 앨런 그린스펀 의장이 지난 2월 ""장기 금리 하락...",ED,0,"['인상/NNG;장기/NNG;금리/NNG;인상/NNG', '국채/NNG;수익률/NN..."
4,4,20050609,[조선일보 제공] 세계 증시의 동조화(同調化)가 갈수록 뚜렷해지고 있다. 주요국들의...,ED,0,"['경제/NNG;성장/NNG;저해/NNG;않/VX', '금리/NNG;경제/NNG;성..."
...,...,...,...,...,...,...
394868,394868,20240711,[이데일리 유준하 기자] 한국은행은 27일 통화안정증권(이하 통안증권) 91일물 경...,ED,0,[]
394869,394869,20240711,[이데일리 정두리 기자] KB국민은행은 서울 코엑스 D홀에서 ‘2024 제1차 KB...,ED,0,"['중견/NNG;기업/NNG;성장/NNG', '성장/NNG;크/VV']"
394870,394870,20240711,[이데일리 유준하 기자] NH투자증권은 27일 과거 미국 연방준비제도(Fed·연준)...,ED,0,"['금리/NNG;인하/NNG;전망/NNG;금리/NNG;인하/NNG', '금리/NNG..."
394871,394871,20240711,[이데일리 이정윤 기자] 원·달러 환율이 1380원 부근까지 상승 출발했다. 소비자...,ED,0,"['금리/NNG;인하/NNG;예상/NNG;인하/NNG', '역외/NNG;환율/NNG..."


In [3]:
df.drop(columns=['content', 'category'], inplace=True)
df.rename(columns={'content_token':'content'}, inplace=True)

In [3]:
import numpy as np
from multiprocessing import Pool
from ekonlpy.sentiment import MPKO

mpko = MPKO(kind=0)

def tokenize_text(text):
    return mpko.tokenize(text)

def parallelize_dataframe(df, func, n_cores=6):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

df['content'] = parallelize_dataframe(df['content'], tokenize_text)

# from ekonlpy.sentiment import MPKO

# mpko = MPKO(kind=0)
# df['content'] = df['content'].apply(lambda x: mpko.tokenize(x))

In [10]:
df

,date,content,rate_change
0,20050609,"['부동산/NNG;가격/NNG;상승/NNG', '가격/NNG;상승/NNG;우호적/V...",0
1,20050609,"['ecb/NNG;금리/NNG;인하/NNG', '경제/NNG;살/VV']",0
2,20050609,"['fed/NNG;금리/NNG;인상/NNG;경고/NNG', 'fed/NNG;장기/N...",0
3,20050609,"['인상/NNG;장기/NNG;금리/NNG;인상/NNG', '국채/NNG;수익률/NN...",0
4,20050609,"['경제/NNG;성장/NNG;저해/NNG;않/VX', '금리/NNG;경제/NNG;성...",0
...,...,...,...
394868,20240711,[],0
394869,20240711,"['중견/NNG;기업/NNG;성장/NNG', '성장/NNG;크/VV']",0
394870,20240711,"['금리/NNG;인하/NNG;전망/NNG;금리/NNG;인하/NNG', '금리/NNG...",0
394871,20240711,"['금리/NNG;인하/NNG;예상/NNG;인하/NNG', '역외/NNG;환율/NNG...",0


In [18]:
import ast

df['content'] = df['content'].apply(lambda x: ast.literal_eval(x))
df = df[df['content'].apply(lambda x: x != [])]
df = df.reset_index(drop=True)

In [21]:
df.to_csv('df_token.csv', index=False)

In [45]:
import csv
import ast
import pandas as pd

df_token = pd.read_csv("df_token.csv")
df_token['content'] = df_token['content'].apply(lambda x: ast.literal_eval(x))
df_token

,date,content,rate_change
0,20050609,"[부동산/NNG;가격/NNG;상승/NNG, 가격/NNG;상승/NNG;우호적/VAX,...",0
1,20050609,"[ecb/NNG;금리/NNG;인하/NNG, 경제/NNG;살/VV]",0
2,20050609,"[fed/NNG;금리/NNG;인상/NNG;경고/NNG, fed/NNG;장기/NNG;...",0
3,20050609,"[인상/NNG;장기/NNG;금리/NNG;인상/NNG, 국채/NNG;수익률/NNG;급...",0
4,20050609,"[경제/NNG;성장/NNG;저해/NNG;않/VX, 금리/NNG;경제/NNG;성장/N...",0
...,...,...,...
359754,20240711,"[소비자/NNG;물가/NNG;상승률/NNG;하락/NNG, 지표/NNG;발표/NNG;...",0
359755,20240711,"[중견/NNG;기업/NNG;성장/NNG, 성장/NNG;크/VV]",0
359756,20240711,"[금리/NNG;인하/NNG;전망/NNG;금리/NNG;인하/NNG, 금리/NNG;인하...",0
359757,20240711,"[금리/NNG;인하/NNG;예상/NNG;인하/NNG, 역외/NNG;환율/NNG;반영...",0


In [46]:
from itertools import chain
from collections import Counter

sample_df = pd.DataFrame()
for tn in range(1, 21) :
    temp_df = pd.DataFrame()
    for i in range(3) :
        temp_df =  pd.concat([temp_df, df_token[df_token['rate_change'] == i].sample(n=40343, random_state=tn)])
    df = temp_df.copy()
    unique_values = set()
    df['content'].apply(lambda x: unique_values.update(set(x)))
    token2idx = {val : idx for idx, val in enumerate(unique_values)}
    count_df = pd.DataFrame(unique_values)
    count_df.rename(columns = {0:'token'}, inplace=True)
    for i in range(3) :
        rc = df[df['rate_change'] == i]
        content_list = list(chain.from_iterable(rc['content'].tolist()))
        content_counter = Counter(content_list)
        count_df[i] = count_df['token'].map(content_counter)
    count_df['fre_sum'] = count_df[0] + count_df[1] + count_df[2]
    count_df['tn'] = tn
    sample_df = pd.concat([sample_df, count_df])
sample_df

,token,0,1,2,fre_sum,tn
0,금리/NNG;오르/VV;부작용/NNG,14,10,6,30,1
1,경기/NNG;부양/NNG;금리/NNG;내리/VV,10,8,24,42,1
2,금리/NNG;성장/NNG;둔화/NNG,0,4,1,5,1
3,투자/NNG;반대/NNG,14,11,12,37,1
4,하락/NNG;경제/NNG;침체/NNG,5,7,6,18,1
...,...,...,...,...,...,...
36495,채권/NNG;금리/NNG;하락/NNG;가격/NNG;상승/NNG,8,3,5,16,20
36496,인플레이션/NNG;우려/NNG;금리/NNG;상승/NNG,8,13,1,22,20
36497,금리/NNG;인상/NNG;불안/NNG;크/VV,1,1,0,2,20
36498,펀더멘털/NNG;자신/NNG,5,6,2,13,20


In [48]:
sample_df.to_csv('sample_unique_count.csv', index=False)

In [47]:
for i in range(1, 21) :
    print(len(sample_df[sample_df['tn'] == i]))

36514
36505
36544
36566
36574
36544
36585
36540
36546
36523
36481
36497
36500
36524
36542
36559
36519
36482
36552
36500


In [4]:
df = sample_df.copy()

In [5]:
(df['content'].apply(len)).sum()

1030681

In [6]:
# unique_values = set()
# df['content'].apply(lambda x: unique_values.update(set(x)))
# token2idx = {val : idx for idx, val in enumerate(unique_values)}

In [7]:
# token2idx

{'금리/NNG;오르/VV;부작용/NNG': 0,
 '경기/NNG;부양/NNG;금리/NNG;내리/VV': 1,
 '금리/NNG;성장/NNG;둔화/NNG': 2,
 '투자/NNG;반대/NNG': 3,
 '하락/NNG;경제/NNG;침체/NNG': 4,
 '개선/NNG;양적완화/NNG': 5,
 '외평채/NNG;스프레드/NNG;안정/NNG': 6,
 '중소기업/NNG;대출/NNG;금리/NNG;낮/VA': 7,
 '하방/NNG;압력/NNG;강하/VA': 8,
 '근원/NNG;인플레이션/NNG;하락/NNG': 9,
 '실물/NNG;경제/NNG;악영향/NNG;우려/NNG': 10,
 '금리/NNG;인플레이션/NNG;압력/NNG': 11,
 '실업자/NNG;늘/VV': 12,
 '아직/MAG;신용위험/NNG': 13,
 '개선/NNG;부진/NNG': 14,
 '금리/NNG;추세/NNG;하향/NNG': 15,
 '역대/NNG;최저/NNG;하락/NNG': 16,
 '인상/NNG;긴축/NNG': 17,
 '경기/NNG;둔화/NNG;양적완화/NNG': 18,
 '제한적/VAX;안정/NNG': 19,
 '식료품/NNG;에너지제외지수/NNG;상승/NNG': 20,
 '자본/NNG;규제/NNG;우려/NNG': 21,
 '물가/NNG;상승률/NNG;물가/NNG;안정/NNG': 22,
 '국채선물/NNG;가격/NNG;고점/NNG': 23,
 '국채선물/NNG;가격/NNG;상승/NNG;줄/VV': 24,
 '국고채/NNG;소폭/MAG;상승/NNG': 25,
 '재정/NNG;성장/NNG': 26,
 '소비자/NNG;물가/NNG;수요/NNG;압력/NNG': 27,
 'fed/NNG;금리/NNG;인상/NNG;늘/VV': 28,
 '신용스프레드/NNG;불안/NNG': 29,
 '지표/NNG;성장/NNG;둔화/NNG': 30,
 '제조업/NNG;생산/NNG;둔화/NNG': 31,
 '외화/NNG;유동성/NNG;확보/NNG': 32,
 '경제/NNG;악재/NNG': 33,
 '수출/NNG;주가/NNG

In [8]:
# count_df = pd.DataFrame(unique_values)
# count_df

,0
0,금리/NNG;오르/VV;부작용/NNG
1,경기/NNG;부양/NNG;금리/NNG;내리/VV
2,금리/NNG;성장/NNG;둔화/NNG
3,투자/NNG;반대/NNG
4,하락/NNG;경제/NNG;침체/NNG
...,...
36500,채권/NNG;금리/NNG;하락/NNG;가격/NNG;상승/NNG
36501,인플레이션/NNG;우려/NNG;금리/NNG;상승/NNG
36502,금리/NNG;인상/NNG;불안/NNG;크/VV
36503,펀더멘털/NNG;자신/NNG


In [9]:
# count_df.rename(columns = {0:'token'}, inplace=True)

In [10]:
# from itertools import chain
# from collections import Counter

# for i in range(3) :
#     rc = df[df['rate_change'] == i]
#     content_list = list(chain.from_iterable(rc['content'].tolist()))
#     content_counter = Counter(content_list)
#     count_df[i] = count_df['token'].map(content_counter)

In [34]:

# count_df.to_csv('token_count.csv', index=False)

In [11]:
#count_df[0] = int(round(count_df[0] * 0.526))  # 극성 1과 2의 신문기사 의 합은 극성 0의 신문기사 개수의 52.67638508710506퍼센트이다. 빈도 수를 맞춰 주기
# count_df

,token,0,1,2
0,금리/NNG;오르/VV;부작용/NNG,7,8,6
1,경기/NNG;부양/NNG;금리/NNG;내리/VV,10,12,20
2,금리/NNG;성장/NNG;둔화/NNG,5,5,1
3,투자/NNG;반대/NNG,9,13,12
4,하락/NNG;경제/NNG;침체/NNG,6,11,6
...,...,...,...,...
36500,채권/NNG;금리/NNG;하락/NNG;가격/NNG;상승/NNG,5,6,5
36501,인플레이션/NNG;우려/NNG;금리/NNG;상승/NNG,8,15,1
36502,금리/NNG;인상/NNG;불안/NNG;크/VV,3,0,0
36503,펀더멘털/NNG;자신/NNG,3,2,2


In [12]:
# count_df['fre_sum'] = count_df[0] + count_df[1] + count_df[2]
# count_df

,token,0,1,2,fre_sum
0,금리/NNG;오르/VV;부작용/NNG,7,8,6,21
1,경기/NNG;부양/NNG;금리/NNG;내리/VV,10,12,20,42
2,금리/NNG;성장/NNG;둔화/NNG,5,5,1,11
3,투자/NNG;반대/NNG,9,13,12,34
4,하락/NNG;경제/NNG;침체/NNG,6,11,6,23
...,...,...,...,...,...
36500,채권/NNG;금리/NNG;하락/NNG;가격/NNG;상승/NNG,5,6,5,16
36501,인플레이션/NNG;우려/NNG;금리/NNG;상승/NNG,8,15,1,24
36502,금리/NNG;인상/NNG;불안/NNG;크/VV,3,0,0,3
36503,펀더멘털/NNG;자신/NNG,3,2,2,7


In [155]:
#for i in range(3) :
#    count_df[str(i) + '_rate'] = count_df[i] / count_df['fre_sum'] * 100

In [13]:
#filtered_df = count_df.copy()

In [157]:
# 성능 개선을 위한 시도
# filtered_df = count_df[(count_df['1_rate'] >= 30) | (count_df['2_rate'] >= 30)]
# filtered_df = filtered_df[abs(filtered_df['1_rate'] - filtered_df['2_rate']) >= 25 ]
# filtered_df.reset_index(drop=True, inplace=True)
# filtered_df.drop(columns=['0_rate', '1_rate', '2_rate'], inplace=True)
# filtered_df

In [162]:
# filtered_df

,token,0,1,2,fre_sum,0_rate,1_rate,2_rate
0,경제정책/NNG;물가/NNG;안정/NNG,2,8,2,12,16.666667,66.666667,16.666667
1,fed/NNG;베이지북/NNG;경제/NNG;성장/NNG,2,2,4,8,25.000000,25.000000,50.000000
2,소득/NNG;부채비율/NNG;높/VA,6,6,1,13,46.153846,46.153846,7.692308
3,미/NNG;국채/NNG;금리/NNG;인플레이션/NNG;우려/NNG,0,4,0,4,0.000000,100.000000,0.000000
4,여전/NNG;물가/NNG;상승/NNG,0,2,0,2,0.000000,100.000000,0.000000
...,...,...,...,...,...,...,...,...
19452,이익률/NNG;상승/NNG,9,8,2,19,47.368421,42.105263,10.526316
19453,채권/NNG;매수/NNG;여력/NNG;감소/NNG,0,0,2,2,0.000000,0.000000,100.000000
19454,가솔린/NNG;재고/NNG;감소/NNG,1,0,1,2,50.000000,0.000000,50.000000
19455,구매/NNG;확대/NNG,12,14,35,61,19.672131,22.950820,57.377049


In [51]:
count_df = sample_df.copy()

In [64]:
prob_df = pd.DataFrame()

k = 0.5
columns = ['0.prob', '1.prob', '2.prob']
for tn in range (1, 21) :
    temp_df = count_df.loc[count_df['tn'] == tn].copy()
    for i in range (3) :
        temp_df[str(i)+'.prob'] = (temp_df[i] + k) / (sum(temp_df[i]) + 2*k)
    p_0 = temp_df[0].sum() / (temp_df['fre_sum'].sum())
    p_1 = temp_df[1].sum() / (temp_df['fre_sum'].sum())
    p_2 = temp_df[2].sum() / (temp_df['fre_sum'].sum())
    temp_df['0.prob'] = temp_df['0.prob'] * p_0
    temp_df['1.prob'] = temp_df['1.prob'] * p_1
    temp_df['2.prob'] = temp_df['2.prob'] * p_2
    for col in columns :
        temp_df[f'Log{col}'] = np.log(temp_df[col])
    prob_df = pd.concat([prob_df, temp_df])

In [66]:
prob_df.reset_index(drop=True, inplace=True)
prob_df

,token,0,1,2,fre_sum,tn,0.prob,1.prob,2.prob,Log0.prob,Log1.prob,Log2.prob
0,금리/NNG;오르/VV;부작용/NNG,14,10,6,30,1,1.404883e-05,0.000010,6.297753e-06,-11.172971,-11.495744,-11.975318
1,경기/NNG;부양/NNG;금리/NNG;내리/VV,10,8,24,42,1,1.017329e-05,0.000008,2.373769e-05,-11.495745,-11.707053,-10.648447
2,금리/NNG;성장/NNG;둔화/NNG,0,4,1,5,1,4.844426e-07,0.000004,1.453328e-06,-14.540267,-12.343042,-13.441655
3,투자/NNG;반대/NNG,14,11,12,37,1,1.404883e-05,0.000011,1.211106e-05,-11.172971,-11.404772,-11.321391
4,하락/NNG;경제/NNG;침체/NNG,5,7,6,18,1,5.328868e-06,0.000007,6.297753e-06,-12.142372,-11.832216,-11.975318
...,...,...,...,...,...,...,...,...,...,...,...,...
730592,채권/NNG;금리/NNG;하락/NNG;가격/NNG;상승/NNG,8,3,5,16,20,8.235819e-06,0.000003,5.329059e-06,-11.707018,-12.594321,-12.142336
730593,인플레이션/NNG;우려/NNG;금리/NNG;상승/NNG,8,13,1,22,20,8.235819e-06,0.000013,1.453380e-06,-11.707018,-11.244394,-13.441619
730594,금리/NNG;인상/NNG;불안/NNG;크/VV,1,1,0,2,20,1.453380e-06,0.000001,4.844599e-07,-13.441619,-13.441619,-14.540231
730595,펀더멘털/NNG;자신/NNG,5,6,2,13,20,5.329059e-06,0.000006,2.422300e-06,-12.142336,-11.975281,-12.930793


In [108]:
prob_df.to_csv('prob_df.csv', index=False)

In [105]:
target_sent = mpc_df['content_token'][20201126]
target_sent

['은행/NNG;가계/NNG;대출/NNG;증가/NNG;확대/NNG',
 '기대/NNG;인플레이션/NNG;소폭/MAG;하락/NNG',
 '투자자금/NNG;유입/NNG;주가/NNG;상승/NNG',
 '가계/NNG;부채/NNG;증가/NNG;금융안정/NNG',
 '원화/NNG;강세/NNG;수출/NNG;부정적/VAX',
 '원달러/NNG;하락/NNG;원화/NNG;강세/NNG',
 '글로벌/NNG;약세/NNG;원화/NNG;강세/NNG',
 '물가/NNG;상승률/NNG;목표/NNG;안정/NNG',
 '경기/NNG;글로벌/NNG;경기/NNG;회복/NNG',
 '경기/NNG;부양책/NNG;기대/NNG;상승/NNG',
 '회복/NNG;물가/NNG;상승/NNG;압력/NNG',
 '성장/NNG;경로/NNG;불확실성/NNG;높/VA',
 '가계/NNG;부채/NNG;증가/NNG;우려/NNG',
 '회복/NNG;설비/NNG;투자/NNG;회복/NNG',
 '완화/NNG;국채/NNG;금리/NNG;상승/NNG',
 '예상/NNG;물가/NNG;상승/NNG;압력/NNG',
 '국제/NNG;유가/NNG;완만/NNG;상승/NNG',
 '경제/NNG;지표/NNG;개선/NNG;상승/NNG',
 '대출/NNG;금리/NNG;소폭/MAG;하락/NNG',
 '기업/NNG;자금/NNG;수요/NNG;증가/NNG',
 '예상/NNG;가능성/NNG;배제/NNG;없/VA',
 '경제/NNG;회복/NNG;기대/NNG;높/VA',
 '경상/NNG;수지/NNG;흑자/NNG;줄/VV',
 '고용동향/NNG;고용자수/NNG;증가/NNG',
 '은행/NNG;대출/NNG;받/VV;어렵/VA',
 '금융시장/NNG;안정/NNG;긍정적/VAX',
 'cds/NNG;프리미엄/NNG;하락/NNG',
 '경제주체/NNG;심리/NNG;개선/NNG',
 '전세가격/NNG;상승/NNG;확대/NNG',
 '전세자금/NNG;대출/NNG;증가/NNG',
 'gdp/NNG;갭/NNG;마이너스/NNG',
 '투자자금/NNG;유입/NNG;확대/NNG',


In [110]:
mpc_df = pd.read_csv('mpc_token.csv')
mpc_df.drop(columns=['Unnamed: 0', 'content'], inplace=True)
mpc_df['content_token'] = mpc_df['content_token'].apply(lambda x: ast.literal_eval(x))
mpc_df = mpc_df.set_index('date')
mpc_df

,content_token
date,
20050609,"[소비자/NNG;물가/NNG;근원/NNG;인플레이션/NNG;안정/NNG, fed/N..."
20050623,"[총액/NNG;한도/NNG;대출/NNG;늘/VV, 금융시장/NNG;안정/NNG, 대..."
20050707,"[소비자/NNG;물가/NNG;근원/NNG;인플레이션/NNG;안정/NNG, 부동산/N..."
20050811,"[소비자/NNG;물가/NNG;근원/NNG;인플레이션/NNG;안정/NNG, 부동산/N..."
20050908,"[금리/NNG;인상/NNG;장기/NNG;금리/NNG;상승/NNG, 인상/NNG;가계..."
...,...
20240523,"[소비/NNG;지출/NNG;pce/NNG;물가/NNG;상승/NNG, fed/NNG;..."
20240613,"[가계/NNG;부채/NNG;질적/VAX;구조/NNG;개선/NNG, 커버드본드/NNG..."
20240626,"[부동산/NNG;주택가격/NNG;하락/NNG, 부동산/NNG;pf/NNG;위험/NN..."


In [123]:
mpc_pol = []
for target_sent in mpc_df['content_token'] :

    pol_count = [0, 0, 0]
    
    for tn in range (1, 21) :
        switch= 0
        temp_df = prob_df.loc[prob_df['tn'] == tn].copy()
        temp_df = temp_df.set_index('token')
        nbc_probs = [np.log(temp_df[col].sum() / temp_df[columns].sum().sum()) for col in columns]
        for token in target_sent:
            for i, col in enumerate(columns):
                try:
                    nbc_probs[i] += temp_df[f'Log{col}'][token]
                    switch = 1
                except KeyError:
                    ...
        if switch == 0 :
            continue
        pol = nbc_probs.index(np.max(nbc_probs))  # 최대 확률 극성값 찾기
        pol_count[pol] += 1
        polarity = pol_count.index(max(pol_count))
    print(pol_count)
    if max(pol_count) == 0 :
        polarity = 4
    mpc_pol.append(polarity)
mpc_df['polarity'] = mpc_pol
mpc_df


[0, 20, 0]
[0, 0, 20]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[0, 0, 20]
[0, 20, 0]
[3, 17, 0]
[0, 0, 20]
[0, 20, 0]
[0, 20, 0]
[4, 16, 0]
[0, 20, 0]
[0, 20, 0]
[0, 0, 0]
[1, 19, 0]
[0, 0, 20]
[0, 20, 0]
[0, 20, 0]
[4, 16, 0]
[0, 0, 0]
[0, 20, 0]
[0, 20, 0]
[2, 18, 0]
[0, 0, 0]
[2, 18, 0]
[0, 0, 20]
[0, 20, 0]
[7, 13, 0]
[0, 0, 0]
[0, 20, 0]
[12, 8, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[1, 19, 0]
[0, 20, 0]
[0, 1, 19]
[0, 20, 0]
[0, 0, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[1, 19, 0]
[0, 20, 0]
[0, 0, 0]
[0, 20, 0]
[0, 0, 0]
[0, 20, 0]
[0, 20, 0]
[0, 0, 0]
[2, 18, 0]
[1, 19, 0]
[0, 20, 0]
[0, 20, 0]
[1, 19, 0]
[0, 0, 0]
[0, 20, 0]
[3, 1, 16]
[0, 20, 0]
[0, 0, 0]
[0, 20, 0]
[0, 20, 0]
[0, 20, 0]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[1, 3, 16]
[0, 0, 20]
[1, 19, 0]
[0, 0, 0]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[0, 0, 20]
[20, 0, 0]
[17, 2, 1]
[0, 0, 0]
[20, 0, 0]
[0, 0, 20]
[19, 1, 0]
[0, 0, 0]
[

,content_token,polarity
date,,
20050609,"[소비자/NNG;물가/NNG;근원/NNG;인플레이션/NNG;안정/NNG, fed/N...",1
20050623,"[총액/NNG;한도/NNG;대출/NNG;늘/VV, 금융시장/NNG;안정/NNG, 대...",2
20050707,"[소비자/NNG;물가/NNG;근원/NNG;인플레이션/NNG;안정/NNG, 부동산/N...",1
20050811,"[소비자/NNG;물가/NNG;근원/NNG;인플레이션/NNG;안정/NNG, 부동산/N...",1
20050908,"[금리/NNG;인상/NNG;장기/NNG;금리/NNG;상승/NNG, 인상/NNG;가계...",1
...,...,...
20240523,"[소비/NNG;지출/NNG;pce/NNG;물가/NNG;상승/NNG, fed/NNG;...",1
20240613,"[가계/NNG;부채/NNG;질적/VAX;구조/NNG;개선/NNG, 커버드본드/NNG...",0
20240626,"[부동산/NNG;주택가격/NNG;하락/NNG, 부동산/NNG;pf/NNG;위험/NN...",2


In [135]:
# 하나씩 찍어보는 용

pol_count = [0, 0, 0]

for tn in range (1, 21) :
    switch= 0
    temp_df = prob_df.loc[prob_df['tn'] == tn].copy()
    temp_df = temp_df.set_index('token')
    nbc_probs = [np.log(temp_df[col].sum() / temp_df[columns].sum().sum()) for col in columns]
    for token in target_sent:
        for i, col in enumerate(columns):
            try:
                nbc_probs[i] += temp_df[f'Log{col}'][token]
                switch = 1
            except KeyError:
                ...
    if switch == 0 :
        continue
    print(nbc_probs)
    pol = nbc_probs.index(np.max(nbc_probs))  # 최대 확률 극성값 찾기
    pol_count[pol] += 1
    polarity = pol_count.index(max(pol_count))
print(pol_count)

[-3544.902986932817, -3635.19226287572, -3594.4214349135327]
[-3525.691769529572, -3593.0274902277156, -3538.890616613596]
[-3572.6183111872697, -3622.9408970310133, -3585.296839260904]
[-3591.0872736090937, -3674.3902453906385, -3619.9859258135466]
[-3599.377373149824, -3661.3440537643587, -3610.744590900648]
[-3534.3498153540495, -3624.8735553949386, -3565.337487874449]
[-3563.673290206775, -3644.085253491775, -3600.6743701076944]
[-3524.99815345098, -3591.099985053588, -3539.845327476586]
[-3565.2348626303315, -3656.6474882358198, -3598.0209029878556]
[-3555.7364327154255, -3618.938975003174, -3571.5506165545285]
[-3540.702702771344, -3625.6420411693116, -3570.924169551094]
[-3572.913975662426, -3627.752329647657, -3587.708916254063]
[-3583.143774618079, -3627.2897798987465, -3586.9590516251506]
[-3534.3712583070505, -3631.389961637208, -3567.0310988317906]
[-3554.843491780046, -3614.7894436212423, -3575.1163832786356]
[-3587.2376070192845, -3670.525391760705, -3621.5602393517324]
[

In [126]:
for i in range(3) : 
    print(len(mpc_df[mpc_df['polarity'] == i]))
print(len(mpc_df[mpc_df['polarity'] == 4]))

123
172
67
28


In [131]:
mpc_df['content_token'][20130613]

['농산물/NNG;제외/NNG;근원/NNG;인플레이션/NNG;상승/NNG',
 '물가/NNG;상승률/NNG;농산물/NNG;가격/NNG;하락/NNG',
 'fed/NNG;자산/NNG;매입/NNG;축소/NNG;우려/NNG',
 '물가/NNG;안정/NNG;목표/NNG;하한/NNG;하회/NNG',
 '가계/NNG;소비/NNG;기업/NNG;투자/NNG;위축/NNG',
 '인하/NNG;기대/NNG;약화/NNG;금리/NNG;상승/NNG',
 '민간/NNG;소비/NNG;설비/NNG;투자/NNG;부진/NNG',
 '금리/NNG;하락/NNG;장기/NNG;금리/NNG;상승/NNG',
 '소비/NNG;투자/NNG;내수/NNG;지표/NNG;개선/NNG',
 '세계/NNG;경제/NNG;완만/NNG;경기/NNG;회복/NNG',
 '저성장/NNG;성장/NNG;잠재력/NNG;훼손/NNG;않/VX',
 'fed/NNG;양적완화/NNG;축소/NNG;우려/NNG',
 '지표/NNG;개선/NNG;fed/NNG;양적완화/NNG',
 'cds/NNG;프리미엄/NNG;소폭/MAG;상승/NNG',
 '자본/NNG;유출입/NNG;변동성/NNG;완화/NNG',
 '중소기업/NNG;대출/NNG;증가/NNG;확대/NNG',
 '소비자/NNG;물가/NNG;상승률/NNG;둔화/NNG',
 '불확실성/NNG;성장/NNG;하방/NNG;위험/NNG',
 '국제/NNG;원자재/NNG;가격/NNG;하향/NNG',
 '전망/NNG;성장률/NNG;전망/NNG;상향/NNG',
 '물가/NNG;상승률/NNG;물가/NNG;안정/NNG',
 '국제/NNG;원자재/NNG;가격/NNG;하락/NNG',
 '유가/NNG;원자재/NNG;가격/NNG;안정/NNG',
 '유로존/NNG;경제/NNG;활동/NNG;부진/NNG',
 '소비자/NNG;물가/NNG;수요/NNG;압력/NNG',
 '가계/NNG;부채/NNG;시스템/NNG;위험/NNG',
 '가격/NNG;소비자/NNG;물가/NNG;상승/NNG',
 '농산물/NNG;제외/NNG;지수/NNG;상승

In [15]:
# p_0 = count_df[0].sum() / (count_df['fre_sum'].sum())
# p_1 = count_df[1].sum() / (count_df['fre_sum'].sum())
# p_2 = count_df[2].sum() / (count_df['fre_sum'].sum())

In [16]:
# count_df['0.prob'] = count_df['0.prob'] * p_0
# count_df['1.prob'] = count_df['1.prob'] * p_1
# count_df['2.prob'] = count_df['2.prob'] * p_2

In [17]:
# prob_df = count_df.copy()

In [18]:
# import numpy as np

# columns = ['0.prob', '1.prob', '2.prob']
# for col in columns :
#     prob_df[f'Log{col}'] = np.log(prob_df[col])
# prob_df = prob_df.set_index('token')
# prob_df

,0,1,2,fre_sum,0.prob,1.prob,2.prob,Log0.prob,Log1.prob,Log2.prob
token,,,,,,,,,,
금리/NNG;오르/VV;부작용/NNG,7,8,6,21,0.000007,8.246952e-06,6.306491e-06,-11.830830,-11.705667,-11.973931
경기/NNG;부양/NNG;금리/NNG;내리/VV,10,12,20,42,0.000010,1.212787e-05,1.988970e-05,-11.494358,-11.320004,-10.825308
금리/NNG;성장/NNG;둔화/NNG,5,5,1,11,0.000005,5.336263e-06,1.455344e-06,-12.140985,-12.140985,-13.440268
투자/NNG;반대/NNG,9,13,12,34,0.000009,1.309810e-05,1.212787e-05,-11.594442,-11.243043,-11.320005
하락/NNG;경제/NNG;침체/NNG,6,11,6,23,0.000006,1.115764e-05,6.306491e-06,-11.973931,-11.403386,-11.973931
...,...,...,...,...,...,...,...,...,...,...
채권/NNG;금리/NNG;하락/NNG;가격/NNG;상승/NNG,5,6,5,16,0.000005,6.306493e-06,5.336262e-06,-12.140985,-11.973931,-12.140985
인플레이션/NNG;우려/NNG;금리/NNG;상승/NNG,8,15,1,24,0.000008,1.503856e-05,1.455344e-06,-11.705667,-11.104893,-13.440268
금리/NNG;인상/NNG;불안/NNG;크/VV,3,0,0,3,0.000003,4.851148e-07,4.851147e-07,-12.592970,-14.538880,-14.538881


In [132]:
target_sent = mpc_df['content_token'][20130613]
target_sent

['농산물/NNG;제외/NNG;근원/NNG;인플레이션/NNG;상승/NNG',
 '물가/NNG;상승률/NNG;농산물/NNG;가격/NNG;하락/NNG',
 'fed/NNG;자산/NNG;매입/NNG;축소/NNG;우려/NNG',
 '물가/NNG;안정/NNG;목표/NNG;하한/NNG;하회/NNG',
 '가계/NNG;소비/NNG;기업/NNG;투자/NNG;위축/NNG',
 '인하/NNG;기대/NNG;약화/NNG;금리/NNG;상승/NNG',
 '민간/NNG;소비/NNG;설비/NNG;투자/NNG;부진/NNG',
 '금리/NNG;하락/NNG;장기/NNG;금리/NNG;상승/NNG',
 '소비/NNG;투자/NNG;내수/NNG;지표/NNG;개선/NNG',
 '세계/NNG;경제/NNG;완만/NNG;경기/NNG;회복/NNG',
 '저성장/NNG;성장/NNG;잠재력/NNG;훼손/NNG;않/VX',
 'fed/NNG;양적완화/NNG;축소/NNG;우려/NNG',
 '지표/NNG;개선/NNG;fed/NNG;양적완화/NNG',
 'cds/NNG;프리미엄/NNG;소폭/MAG;상승/NNG',
 '자본/NNG;유출입/NNG;변동성/NNG;완화/NNG',
 '중소기업/NNG;대출/NNG;증가/NNG;확대/NNG',
 '소비자/NNG;물가/NNG;상승률/NNG;둔화/NNG',
 '불확실성/NNG;성장/NNG;하방/NNG;위험/NNG',
 '국제/NNG;원자재/NNG;가격/NNG;하향/NNG',
 '전망/NNG;성장률/NNG;전망/NNG;상향/NNG',
 '물가/NNG;상승률/NNG;물가/NNG;안정/NNG',
 '국제/NNG;원자재/NNG;가격/NNG;하락/NNG',
 '유가/NNG;원자재/NNG;가격/NNG;안정/NNG',
 '유로존/NNG;경제/NNG;활동/NNG;부진/NNG',
 '소비자/NNG;물가/NNG;수요/NNG;압력/NNG',
 '가계/NNG;부채/NNG;시스템/NNG;위험/NNG',
 '가격/NNG;소비자/NNG;물가/NNG;상승/NNG',
 '농산물/NNG;제외/NNG;지수/NNG;상승

In [34]:
nbc_probs = [np.log(prob_df[col].sum() / prob_df[columns].sum().sum()) for col in columns]
nbc_probs

[-1.13888990414344, -1.027962173962537, -1.1329226212426884]

In [133]:
nbc_probs = [-1.13888990414344, -1.027962173962537, -1.1329226212426884]

In [35]:
#target_sent = ['근원/NNG;인플레이션/NNG;소비자/NNG;물가/NNG;안정/NNG', '국제/NNG;유가/NNG;원자재/NNG;가격/NNG;상승/NNG', 'cd/NNG;금리/NNG;상승/NNG;금리/NNG;인상/NNG', '경상/NNG;수지/NNG;상품/NNG;수지/NNG;흑자/NNG', '근원/NNG;인플레이션/NNG;물가/NNG;안정/NNG', 'fed/NNG;금리/NNG;인상/NNG;종료/NNG', 'fed/NNG;금리/NNG;인상/NNG;중단/NNG', '하락/NNG;fed/NNG;금리/NNG;인상/NNG', '가격/NNG;소비자/NNG;물가/NNG;상승/NNG', 'fed/NNG;금리/NNG;인상/NNG;종결/NNG', '금리/NNG;fed/NNG;금리/NNG;인상/NNG', '금리/NNG;ecb/NNG;금리/NNG;인상/NNG', '예상/NNG;fed/NNG;금리/NNG;인상/NNG', '우려/NNG;금리/NNG;인상/NNG;중단/NNG', '금리/NNG;인상/NNG;중단/NNG;약세/NNG', '기대/NNG;반영/NNG;금리/NNG;인상/NNG', '금리/NNG;인상/NNG;환율/NNG;상승/NNG', '금리/NNG;인상/NNG;환율/NNG;하락/NNG', '물가/NNG;국제/NNG;유가/NNG;상승/NNG', '원화/NNG;절상/NNG;금리/NNG;인상/NNG', '물가/NNG;상승/NNG;압력/NNG;억제/NNG', '재정/NNG;경상/NNG;수지/NNG;적자/NNG', '물가/NNG;안정/NNG;경제/NNG;성장/NNG', '금리/NNG;인상/NNG;원화/NNG;절상/NNG', '금리/NNG;인상/NNG;전망/NNG;상승/NNG', '경상/NNG;수지/NNG;흑자/NNG;축소/NNG', '금리/NNG;인상/NNG;예상/NNG;상승/NNG', '기업/NNG;실적/NNG;부진/NNG;하락/NNG', '콜/NNG;금리/NNG;목표/NNG;인상/NNG', '콜/NNG;금리/NNG;목표/NNG;상향/NNG', '인플레이션/NNG;목표/NNG;하회/NNG', '수출기업/NNG;채산성/NNG;악화/NNG', '경제/NNG;작/VV;않/VX;부담/NNG', '근원/NNG;인플레이션/NNG;상승/NNG', '거주자/NNG;외화예금/NNG;감소/NNG', '금융시장/NNG;금리/NNG;인상/NNG', '채권투자/NNG;심리/NNG;회복/NNG', '통화/NNG;완화정책/NNG;종결/NNG', '수출/NNG;증가율/NNG;둔화/NNG', '당분간/MAG;금리/NNG;인상/NNG', '부동산/NNG;가격/NNG;불안/NNG', '경제/NNG;성장률/NNG;둔화/NNG', '지난번/NNG;금리/NNG;인상/NNG', '부동산/NNG;투기/NNG;억제/NNG', '기업/NNG;생산성/NNG;향상/NNG', '부동산/NNG;가격/NNG;안정/NNG', '고유가/NNG;상승/NNG;압력/NNG', '둔화/NNG;원화/NNG;절상/NNG', '자본/NNG;수지/NNG;흑자/NNG', '외환시장/NNG;영향/NNG;줄/VV', '원화/NNG;절상/NNG;압력/NNG', '금리/NNG;과도/NNG;상승/NNG', '금리/NNG;인상/NNG;상승/NNG', '하락/NNG;물가/NNG;안정/NNG', '주택/NNG;경기/NNG;둔화/NNG', '소비/NNG;회복/NNG;지연/NNG', '통화/NNG;절상/NNG;압력/NNG', '수출/NNG;견조/NNG;증가/NNG', '과도/NNG;물가/NNG;상승/NNG', '경제/NNG;안정/NNG;성장/NNG', '회복/NNG;물가/NNG;안정/NNG', '국제/NNG;유가/NNG;불안/NNG', '내수/NNG;경기/NNG;회복/NNG', '악화/NNG;경기/NNG;회복/NNG', '대출/NNG;금리/NNG;인상/NNG', '수출/NNG;증가/NNG;회복/NNG', '수출/NNG;증가/NNG;확대/NNG', '국채/NNG;발행/NNG;확대/NNG', '곡물/NNG;가격/NNG;안정/NNG', '곡물/NNG;가격/NNG;급등/NNG', '절상/NNG;환율/NNG;하락/NNG', '절상/NNG;원화/NNG;절상/NNG', '수출/NNG;수입/NNG;감소/NNG', '원화/NNG;환율/NNG;하락/NNG', '단기/NNG;금리/NNG;상승/NNG', '신용/NNG;공급/NNG;확대/NNG', '자금/NNG;수요/NNG;증가/NNG', '금리/NNG;하향/NNG;안정/NNG', '개선/NNG;환율/NNG;하락/NNG', '금리/NNG;인상/NNG;그치/VV', '예상/NNG;강하/VA;회복/NNG', '중단/NNG;가능성/NNG;높/VA', '아직/MAG;우려/NNG;않/VX', '제로금리정책/NNG;종결/NNG', '인플레이션/NNG;하락/NNG', '외환보유액/NNG;증가/NNG', '외환시장/NNG;안정/NNG', '불확실성/NNG;확산/NNG', '실효환율/NNG;하락/NNG', '불확실성/NNG;감소/NNG', '통화정책/NNG;완화/NNG', '불안정성/NNG;증대/NNG', '미래/NNG;불확실성/NNG', '중소기업/NNG;도산/NNG', '공공요금/NNG;인상/NNG', '실효환율/NNG;절상/NNG', '자금사정/NNG;개선/NNG', '절상/NNG;부정적/VAX', '상승률/NNG;상회/NNG', '부동산/NNG;불안/NNG', '원자재/NNG;안정/NNG', '부동산/NNG;안정/NNG', '수출/NNG;일시적/VAX', '일시적/VAX;감소/NNG', '원달러/NNG;하락/NNG', '연체율/NNG;하향/NNG', '투자/NNG;부진/NNG', '실적/NNG;저조/NNG', '절상/NNG;추세/NNG', '수출/NNG;둔화/NNG', '자금/NNG;흡수/NNG', '경기/NNG;비관/NNG', '투자금액/NNG;많/VA', '불안/NNG;심각/NNG', '심각/NNG;우려/NNG', '금리/NNG;위험/NNG', '투기/NNG;거품/NNG', '억제/NNG;효과/NNG', '절상/NNG;효과/NNG', '수출/NNG;위축/NNG', '위축/NNG;우려/NNG', '회복/NNG;하향/NNG', '불안/NNG;해소/NNG', '상방/NNG;위험/NNG', '완화/NNG;축소/NNG', '안정/NNG;위협/NNG', '정치/NNG;불안/NNG', '우려/NNG;심각/NNG', '경제/NNG;축소/NNG', '예상/NNG;상회/NNG', '상회/NNG;안정/NNG', '안정/NNG;상승/NNG', '투자/NNG;우려/NNG', '불안정성/NNG;높/VV', '수요/NNG;압력/NNG', '비슷/NNG;성장/NNG', '경제/NNG;회복/NNG', '우려/NNG;아니/VCN', '하락/NNG;한계/NNG', '견실/NNG;성장/NNG', '판매/NNG;감소/NNG', '둔화/NNG;부진/NNG', '수입/NNG;급증/NNG', '하락/NNG;절상/NNG', '적자/NNG;감소/NNG', '외채/NNG;증가/NNG', '공급/NNG;증가/NNG', '금리/NNG;인하/NNG', '개선/NNG;못하/VX', '수요/NNG;오르/VV', '부진/NNG;않/VX', '높/VA;증가/NNG', '전망/NNG;높/VA', '투자/NNG;늘/VV', '경제/NNG;크/VV', '수요/NNG;늘/VV', '인상/NNG;좋/VA', '예상/NNG;높/VA', '높/VA;회복/NNG', '위험/NNG;높/VA', '높/VA;상승/NNG', '수출/NNG;높/VA', '유가상승/NNG', '부작용/NNG', '불안정/NNG', '고성장/NNG', '호전/NNG', '확장/NNG', '자제/NNG', '지장/NNG', '최저/NNG', '양호/NNG', '폐쇄/NNG', '한파/NNG', '급락/NNG']
import numpy as np

x = 0
for token in target_sent:
    for i, col in enumerate(columns):
        try:
            nbc_probs[i] += prob_df[f'Log{col}'][token]
            x += 1
            print(token)
            print(prob_df[f'Log{col}'][token])
            print()
        except KeyError:
            # nbc_probs[i] += np.log(0.5)
            ...
#np.set_printoptions(precision=10, suppress=True)
print(nbc_probs)
#print(np.exp(nbc_probs) / np.exp(nbc_probs).sum())
# max_nbc_prob = np.max(nbc_probs)  # 안정성을 위한 최대 값 찾기
# log_probs = nbc_probs - max_nbc_prob  # 수치적 안정성을 위한 조정
# exp_probs = np.exp(log_probs)  # 로그 스케일에서 지수 계산
# sum_exp_probs = exp_probs.sum()  # 총합 계산

# # 확률로 변환
# normalized_probs = exp_probs / sum_exp_probs

# np.set_printoptions(precision=10, suppress=True)
# print(normalized_probs)
print(x)


소비자/NNG;물가/NNG;농축수산물/NNG;가격/NNG;상승/NNG
-12.929442611859004

소비자/NNG;물가/NNG;농축수산물/NNG;가격/NNG;상승/NNG
-12.592970040641454

소비자/NNG;물가/NNG;농축수산물/NNG;가격/NNG;상승/NNG
-14.538880505229523

소비자/NNG;물가/NNG;상승률/NNG;목표/NNG;밑돌/VV
-11.973931166831568

소비자/NNG;물가/NNG;상승률/NNG;목표/NNG;밑돌/VV
-12.592970040641454

소비자/NNG;물가/NNG;상승률/NNG;목표/NNG;밑돌/VV
-14.538880505229523

국고채/NNG;금리/NNG;상승/NNG;금리/NNG;상승/NNG
-11.320004699424903

국고채/NNG;금리/NNG;상승/NNG;금리/NNG;상승/NNG
-11.042372628230288

국고채/NNG;금리/NNG;상승/NNG;금리/NNG;상승/NNG
-10.334187885838558

수요측면/NNG;물가/NNG;상승/NNG;압력/NNG;않/VX
-11.320004699424903

수요측면/NNG;물가/NNG;상승/NNG;압력/NNG;않/VX
-11.973930832235231

수요측면/NNG;물가/NNG;상승/NNG;압력/NNG;않/VX
-13.440268216561414

경제/NNG;지표/NNG;개선/NNG;경기/NNG;회복/NNG
-11.705667180236889

경제/NNG;지표/NNG;개선/NNG;경기/NNG;회복/NNG
-14.538880189696767

경제/NNG;지표/NNG;개선/NNG;경기/NNG;회복/NNG
-13.440268216561414

국채선물/NNG;순매도/NNG;영향/NNG;상승/NNG
-12.59297037523779

국채선물/NNG;순매도/NNG;영향/NNG;상승/NNG
-14.538880189696767

국채선물/NNG;순매도/NNG;영향/NNG;상승/NNG
-13.4402

In [105]:
prob_df

,0,1,2,fre_sum,0.prob,1.prob,2.prob,Log0.prob,Log1.prob,Log2.prob
token,,,,,,,,,,
금리/NNG;변동/NNG;취약/NNG,15,9,2,26,2.925500e-05,0.000018,4.718469e-06,-10.439460,-10.929010,-12.264026
중장기물/NNG;금리/NNG;상승/NNG;확대/NNG,0,1,0,1,9.437098e-07,0.000003,9.436937e-07,-13.873447,-12.774836,-13.873464
예상/NNG;상회/NNG;상승/NNG,39,20,3,62,7.455308e-05,0.000039,6.605856e-06,-9.503999,-10.159877,-11.927554
주택/NNG;지표/NNG;둔화/NNG,11,14,0,25,2.170533e-05,0.000027,9.436937e-07,-10.737953,-10.506153,-13.873464
인플레이션/NNG;우려/NNG;높/VA;않/VX,4,3,1,8,8.493389e-06,0.000007,2.831081e-06,-11.676223,-11.927538,-12.774852
...,...,...,...,...,...,...,...,...,...,...
전망/NNG;국채/NNG;발행/NNG;증가/NNG,4,2,0,6,8.493389e-06,0.000005,9.436937e-07,-11.676223,-12.264011,-13.873464
낙찰/NNG;금리/NNG;유통금리/NNG;높/VA,2,3,1,6,4.718549e-06,0.000007,2.831081e-06,-12.264009,-11.927538,-12.774852
하락/NNG;경기/NNG;모멘텀/NNG;약화/NNG,2,2,0,4,4.718549e-06,0.000005,9.436937e-07,-12.264009,-12.264011,-13.873464
